In [1]:
import pandas as pd
import numpy as np

import os

In [3]:
basics_url="https://datasets.imdbws.com/title.basics.tsv.gz"

akas_url="https://datasets.imdbws.com/title.akas.tsv.gz"

ratings_url = "https://datasets.imdbws.com/title.ratings.tsv.gz"

In [4]:
basics = pd.read_csv(basics_url, sep='\t', low_memory=False)

In [5]:
akas = pd.read_csv(akas_url, sep='\t', low_memory=False)

In [6]:
ratings = pd.read_csv(ratings_url, sep='\t', low_memory=False)

Filtering/Cleaning Steps:

Title Basics:
- Replace "\N" with np.nan
- Eliminate movies that are null for runtimeMinutes
- Eliminate movies that are null for genre
- keep only titleType==Movie
- keep startYear 2000-2022
- Eliminate movies that include "Documentary" in genre (see tip below)
- Keep only US movies (Use AKAs table, see "Filtering one dataframe based on another" section below)

In [7]:
# replacing /N with NaN
basics = basics.replace({'\\N':np.nan})
akas = akas.replace({'\\N':np.nan})
ratings = ratings.replace({'\\N':np.nan})

In [8]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9221340 entries, 0 to 9221339
Data columns (total 9 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   tconst          object
 1   titleType       object
 2   primaryTitle    object
 3   originalTitle   object
 4   isAdult         object
 5   startYear       object
 6   endYear         object
 7   runtimeMinutes  object
 8   genres          object
dtypes: object(9)
memory usage: 633.2+ MB


In [14]:
#removing entries missing Run Time and Genre
basics = basics.dropna(axis=0, subset=['runtimeMinutes','genres'])
basics.shape

(367884, 9)

In [15]:
basics['titleType'].unique()

array(['movie'], dtype=object)

In [12]:
#Filtering Title Type
basics = basics.loc[basics['titleType'] == 'movie']
basics.shape

(367884, 9)

In [17]:
# preparing columns to filter Year, so converting obj into int after removing NaN
basics['startYear'] = basics['startYear'].fillna(0)


In [18]:
basics['startYear'] = basics['startYear'].astype('int64')


In [19]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 367884 entries, 8 to 9221290
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   tconst          367884 non-null  object
 1   titleType       367884 non-null  object
 2   primaryTitle    367884 non-null  object
 3   originalTitle   367884 non-null  object
 4   isAdult         367884 non-null  object
 5   startYear       367884 non-null  int64 
 6   endYear         0 non-null       object
 7   runtimeMinutes  367884 non-null  object
 8   genres          367884 non-null  object
dtypes: int64(1), object(8)
memory usage: 28.1+ MB


In [20]:
#this columns is missing all values
basics.drop(columns='endYear', inplace=True)

In [21]:
# Taking movies from a period of time
basics = basics.loc[(basics['startYear'] >= 2000) & (basics['startYear'] <= 2022)]
basics.shape

(215767, 8)

In [22]:
basics['startYear'].unique()

array([2021, 2001, 2020, 2006, 2018, 2005, 2004, 2002, 2009, 2017, 2000,
       2008, 2007, 2003, 2022, 2012, 2010, 2013, 2019, 2011, 2015, 2016,
       2014], dtype=int64)

In [23]:
# Exclude movies that are included in the documentary category.
is_documentary = basics['genres'].str.contains('documentary',case=False)
basics = basics[~is_documentary]



AKAs:
- keep only US movies.
- Replace "\N" with np.nan

In [24]:
akas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33173231 entries, 0 to 33173230
Data columns (total 8 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   titleId          object
 1   ordering         int64 
 2   title            object
 3   region           object
 4   language         object
 5   types            object
 6   attributes       object
 7   isOriginalTitle  object
dtypes: int64(1), object(7)
memory usage: 2.0+ GB


array(['UA', 'DE', 'HU', 'GR', 'RU', 'US', nan, 'JP', 'FR', 'RO', 'GB',
       'CA', 'PT', 'ES', 'FI', 'PL', 'AR', 'RS', 'UY', 'IT', 'BR', 'DK',
       'TR', 'XWW', 'XEU', 'SK', 'CZ', 'SE', 'NZ', 'MX', 'NO', 'XYU',
       'AT', 'VE', 'CSHH', 'SI', 'IN', 'AU', 'TW', 'LT', 'NL', 'CO', 'IR',
       'BG', 'SG', 'BE', 'SUHH', 'VN', 'HR', 'DZ', 'CH', 'BF', 'PH',
       'XWG', 'CN', 'XSA', 'EE', 'IS', 'PR', 'DDDE', 'HK', 'XKO', 'CL',
       'IE', 'JM', 'PE', 'EG', 'GE', 'BY', 'KR', 'BA', 'AE', 'PA', 'TJ',
       'XSI', 'TH', 'YUCS', 'ZA', 'MY', 'IL', 'LV', 'ID', 'PK', 'BD',
       'CU', 'AL', 'BO', 'XAS', 'CR', 'PY', 'DO', 'GT', 'EC', 'SV', 'UZ',
       'BUMM', 'XPI', 'BJ', 'AZ', 'NG', 'CM', 'MA', 'GL', 'MN', 'LI',
       'LU', 'MZ', 'BM', 'KZ', 'MD', 'LB', 'IQ', 'TM', 'MK', 'TN', 'HT',
       'AM', 'LK', 'ME', 'CG', 'CI', 'SY', 'NP', 'QA', 'TO', 'SN', 'GH',
       'JO', 'KG', 'NE', 'GN', 'VDVN', 'TD', 'SO', 'SD', 'MC', 'TT', 'GA',
       'BS', 'LY', 'AO', 'KH', 'MR', 'AF', 'MG', 'ML', 'GY', 

In [25]:
#Filtering US movies
akas = akas.loc[akas['region']=='US']
akas.shape

(1348747, 8)

Ratings:
- Replace "\N" with np.nan (if any)
- Keep only US movies (Use AKAs table, see "Filtering one dataframe based on another" section below)


In [26]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1260427 entries, 0 to 1260426
Data columns (total 3 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   tconst         1260427 non-null  object 
 1   averageRating  1260427 non-null  float64
 2   numVotes       1260427 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 28.8+ MB


In [27]:
# Filter the basics table down to only include the US by using the filter akas dataframe 

#For Basics and Ratings
keepers =basics['tconst'].isin(akas['titleId'])
basics = basics[keepers]

#doing it without saving filter in a variable

ratings =ratings[ratings['tconst'].isin(akas['titleId'])]

print(ratings.shape)
print(basics.shape)



(476236, 3)
(82476, 8)


CHECKING

In [28]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 82476 entries, 34792 to 9221105
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   tconst          82476 non-null  object
 1   titleType       82476 non-null  object
 2   primaryTitle    82476 non-null  object
 3   originalTitle   82476 non-null  object
 4   isAdult         82476 non-null  object
 5   startYear       82476 non-null  int64 
 6   runtimeMinutes  82476 non-null  object
 7   genres          82476 non-null  object
dtypes: int64(1), object(7)
memory usage: 5.7+ MB


In [29]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 476236 entries, 0 to 1260423
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   tconst         476236 non-null  object 
 1   averageRating  476236 non-null  float64
 2   numVotes       476236 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 14.5+ MB


In [ ]:
akas.info()

SAVING

In [95]:
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres
34792,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,118,"Comedy,Fantasy,Romance"
61093,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020,70,Drama
67639,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,122,Drama
86770,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,100,"Comedy,Horror,Sci-Fi"
93906,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002,126,Drama


In [97]:
#making new folder with os

os.makedirs('Data/',exist_ok=True) 
# Confirm folder created
os.listdir("Data/")


[]

In [98]:
## Save current dataframe to file.
basics.to_csv("Data/title_basics.csv.gz",compression='gzip',index=False)


In [99]:
akas.to_csv("Data/title_akas.csv.gz",compression='gzip',index=False)

In [100]:
ratings.to_csv("Data/title_ratings.csv.gz",compression='gzip',index=False)

In [102]:
# Open saved file and preview again
basics = pd.read_csv("Data/title_basics.csv.gz", low_memory = False)
akas = pd.read_csv("Data/title_akas.csv.gz", low_memory = False)
ratings = pd.read_csv("Data/title_ratings.csv.gz", low_memory = False)

basics.head()



,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,122,Drama
3,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,100,"Comedy,Horror,Sci-Fi"
4,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002,126,Drama
